In [222]:
import sympy

# System is a vector of differential equations

system = [
    "f'=w",
    "g'=-p*q*v-22.1",
    "w'=-w+u+v",
    "u'=-u+rv",
    "v'=-v",
    "r'=-r**2",
    "p'=p*v",
    "q'=v-q"
]

# Init is a vector representing the system's initial condition

init = [0, 0, 0, 0, 0, 1, 1, 1]

In [223]:
# Utilities for handling the initial conditions

def split_eq(equation: str) -> list[str]:
    return equation.split("=")


def extract_dv(eq: str) -> str:
    lhs = split_eq(equation=eq)[0]
    if len(lhs) == 1: 
        return lhs
    if lhs.endswith("_prime"):
        parts = lhs.split("_prime")
        dv = ""
        for part in parts[:-1]:
            dv += part
        return dv
    if lhs.endswith("'"):
        return lhs.split("'")[0]

def rewrite_eq_with_init(eq: str, init: float) -> str: 
    parts = split_eq(equation=eq)
    lhs = parts[0]
    rhs = parts[1]
    dv = extract_dv(eq=eq)
    replace_with = dv
    if init > 0:
        replace_with += f"+{init}"
    elif init < 0:
        replace_with += f"-{init}"
    replace_with = f"({replace_with})"
    rhs = rhs.replace(dv, replace_with)
    result = lhs + "=" + rhs
    return result 

In [224]:
# Expression parsing utilities

import sympy

def parse_rhs(expr: str):
    return sympy.parse_expr(expr)

In [235]:
import sys

class CrnEquation:
    """
    Expects a single equation and its numeric initial condition
    """
    def __init__(self, eq: str, init: float):
        self.original = eq
        if init == 0:
            self.adjusted = eq
        else:
            self.adjusted = rewrite_eq_with_init(eq=eq, init=init)
        self.dv = extract_dv(eq=eq)
        self.parts = self.adjusted.split("=")
        self.crn_computable = self.crn_computable()

    def crn_computable(self) -> bool:
        rhs = self.parts[1]
        rhs = sympy.parse_expr(rhs)
        pos, neg = self.split_rhs(rhs)

        # CONDITION ONE 
        # Determine whether a negative constant exists. If one does, the expression is not CRN computable
        for el in neg:
            match type(el):
                case sympy.core.numbers.Integer:
                    cast = int(el)
                    if cast < 0:
                        return False
                case sympy.core.numbers.Float:
                    cast = float(el)
                    if cast < 0:
                        return False
         
        # CONDITION TWO
        # Determine whether ALL expressions with negative terms on the RHS contain the dependent variable. If not, the expression is
        # not CRN computable 

        return True 

    def split_rhs(self, rhs):
        ispos = lambda x: x.as_coeff_Mul()[0].is_positive
        pos, neg = sympy.sift(sympy.Add.make_args(rhs), ispos, binary=True)
        return pos, neg 

    def __str__(self):
        return f"""
CrnEquation [ 
    original: {self.original},
    dv: {self.dv}
    adjusted: {self.adjusted},
    parts: {self.parts},
    crn_computable: {self.crn_computable}
]"""

    def __repr__(self):
        return str(self) 

class Crn:
    """
    Expects a vector of differential equations and an equal-sized vector of initial conditions
    """
    def __init__(self, system: list[str], init: list[float]):
        self.original_system = system
        self.system: list[CrnEquation] = []
        self.init = init
        self.rewrite_queue: list[str] = []
        if len(self.original_system) != len(self.init):
            sys.exit(1)
        self.handle_initial_values()
        

    def handle_initial_values(self): 
        for eq, init in zip(self.original_system, self.init):
            crn_equation = CrnEquation(eq=eq, init=init)
            self.system.append(crn_equation)

In [236]:
crn = Crn(system=system, init=init)

-22.1
